# How to visualize your VNET topology in Azure with PowerShell

> This notebook does work on Linux but does not work in Binder for some reason :(

## Scenario

Assume we have some network topology in Azure. The topology is dynamic, VNETs being added and removed dynamically, and at any particular moment, we can't say how it looks without exploring it. We want to visualize the topology somehow, at any point in time. Additionally, we want to highlight VNETs, where VNET peering was not configured correctly, as we know that this kind of thing happens from time to time due to some bugs in automated procedures we have. In addition to this, we also know that there are a few devices connected to some networks. Someone complained that one of these devices does not work as expected, and we want to see where it is

## Preparation

In this section we are going to prepare a lab, where we going to run a few experiments.

Before everything, we need to install a few modules, just in case. This step is also useful in Binder, where there are no such modules

In [ ]:
Install-Module -Name PSQuickGraph -AllowPrerelease -RequiredVersion "2.0.2-alpha"
Install-Module -Name ipmgmt
Install-Module -Name Az -Scope CurrentUser -Repository PSGallery

And load them

In [ ]:
Import-Module ipmgmt
Import-Module PSQuickGraph -RequiredVersion "2.0.2"

Log in to Azure. It will fail in Binder ...

In [ ]:
Login-AzAccount

and it will ask you to use this

In [ ]:
Connect-AzAccount -UseDeviceAuthentication

set the initial constants

In [ ]:
$rgName = "vnet-test"
$region = "eastus2"
$baseNet = "10.96.0.0/16"

Create the resource group for our experiment, just in case.

In [ ]:
New-AzResourceGroup -Name $rgName -Location $region

Now, we can prepare a set of parameters for VNETs. We are using the `ipmgmt`, which we looked at in the [previous](https://github.com/eosfor/scripting-notes/blob/main/notebooks/ru/ipmgmt.ipynb) notebook. Here we simply make an array of hashtables and pass it to the `Get-VLSMBreakdown` cmdlet, which returns us the IP ranges we are looking for. We just iterate from "A" to "K" and for each character create an element in this array.

In [ ]:
$vnets = "A".."K" | % { @{type = "VNET-$_"; size = (256-2)} }

Now we just pass the array to the commandlet, and it returns a list of ranges we need.

In [ ]:
Get-VLSMBreakdown -Network $baseNet -SubnetSize $vnets | ? type -ne 'reserved' | 
    ft type, network, netmask, *usable, cidr -AutoSize

At this point, we can create these networks. To simplify, we create only one subnet in each, but nothing stops us from using `Get-VLSMBreakdown` to break each VNET into subnets if we need to.

In [ ]:
Get-VLSMBreakdown -Network $baseNet -SubnetSize $vnets | ? type -ne 'reserved' | % {
    $addressPrefix = "$($_.network)/$($_.cidr)"
    $subnet = New-AzVirtualNetworkSubnetConfig -Name "default" -AddressPrefix $addressPrefix
    New-AzVirtualNetwork -Name $_.type -ResourceGroupName $rgName -Location $region -AddressPrefix $addressPrefix -Subnet $subnet | out-null
}

Adding VNET peering

In [ ]:
$nets = Get-AzVirtualNetwork -ResourceGroupName $rgName

We are going to use hub-and-spoke topology, as we usually do in Landing Zones. And to simulate a misconfiguration, we skip one VNET peering link

In [ ]:
$hub = $nets[0]
$spokes = $nets[1..($nets.count-1)]

$spokes | % {
    Add-AzVirtualNetworkPeering `
        -Name "$($hub.Name)-$($_.Name)" `
        -VirtualNetwork $hub `
        -RemoteVirtualNetworkId $_.Id | out-null
}

$spokes | select -Skip 1 |  % {      
    Add-AzVirtualNetworkPeering `
        -Name "$($_.Name)-$($hub.Name)" `
        -VirtualNetwork $_ `
        -RemoteVirtualNetworkId $hub.Id | out-null
}

To model the network devices we add a few NICs into random VNETs. This is a lab, and we don't need full VMs for that. Well, in this case for the sake of the experiment we need a NIC to exist in a specific VNET. For that we use `begin`, `process`, and `end` script block parameters of the `Foreach-Object`. In this case, we initialize the variable in the `begin` script block and use it in the `process`, so it is guaranteed to be initialized with the correct value.

In [ ]:
1..4 | % {$idx = 0} {
    $vnetForNIC = $spokes[$idx]
    New-AzNetworkInterface -Name "NetworkInterface-$idx" -ResourceGroupName $rgName -Location $region -SubnetId $vnetForNIC.Subnets[0].Id
    $idx = Get-Random -Minimum 1 -Maximum ($spokes.Count-1)
}

## Experiment

If all the above worked well, we are ready for the experiment. Let us read everything again, and create an empty graph

In [ ]:
$vnets = Get-AzVirtualNetwork -ResourceGroupName $rgName
$g = New-Graph

We need to fill the graph with data, and we are going to do it in a few iterations. At fist, we will add VNETs, then VNET peerings, and at the end - NICs

#### Adding VNETs

We have everything we need to add VNETs to the graph. However, there is a little problem. We can add any objects to the graph, but when we do it, the graph "converts" it into a special type - `PSGraph.Model.PSVertex`. There is the `Label` property in this type, and it is a key property for comparison operations, search and visualization. Another property of interest is the `OriginalObject` property. It has a link to the origunal object, which was used to create the vertex. To initialize the `Label` property the `ToString()` method is being called on each object. In cas of `PSVirtualNetwork` this call returns the name of the type, but we need unique names as labels. To overcome this we have a constructor for the `PSGraph.Model.PSVertex` whch take as `Label` as `string` and the original object. This way we can override the labeling mechanism and provide our own labels.

In [ ]:
$vnets | % {
    Add-Vertex -Graph $g -Vertex ([PSGraph.Model.PSVertex]::new($_.Id, $_))
}

#### Adding VNET peerings

At this point, we have VNETs in the graph. Now we need to add connections between these networks. To do that we iterate through the graph vertices, pull the `OriginalObject` property of each of them, and peek into it to find VNET's peerings. Each peering stores the `resourceID` of the remote VNET. As far as we use VNET resource IDs as labels on the graph, we can just search the graph for the corresponding vertex, and use it as a target for the graph edge. So the source is the current VNET, and the target is the VNET pointed by the peering information.

In [ ]:
foreach ($v in $g.Vertices){
    foreach($p in $v.OriginalObject.VirtualNetworkPeerings) {
        foreach ($rvn in $p.RemoteVirtualNetwork) {
            $targetVertex = $g.Vertices.Where({$_.Label -eq $rvn.id})[0]
            Add-Edge -From $v -To $targetVertex -Graph $g
        }
    }
}

Now we have networks and links between them. So we can visualize the topology. But we want to highlight misconfigured networks. Here we cheat a bit because we know how exactly the network was misconfigured. In the real scenario, you would test for more than one misconfigured thing. But in our case, just for simplicity, we are running through vertices again, and colorizing those which does not have outgoing edges.

In [ ]:
$g.Vertices | % { if ( $g.OutDegree($_) -eq 0 ) 
                    { $_.GVertexParameters.Fillcolor = [QuikGraph.Graphviz.Dot.GraphvizColor]::OrangeRed  } 
                }

#### Adding NICs

The last step we need to to is to add NICs to our graph. We read them first

In [ ]:
$nics = Get-AzNetworkInterface -ResourceGroupName $rgName

Each NIC has a reference to its subnet in the `IpConfigurations` property, stored as a `resourceID`. It is a string, so we can just cut everything after `/subnets/` and use the rest of the string as a reference to a VNET.

In [ ]:
$nics | % {
    $vnetID = $_.IpConfigurations[0].Subnet.Id -replace "/subnets/.+", ""
    $targetVertex = $g.Vertices.Where({$_.Label -eq $vnetID})[0]
    Add-Edge -Graph $g -From ([PSGraph.Model.PSVertex]::new($_.name, $_)) -To $targetVertex
}

Now, we have everything we need in the graph. As the last step we want to color **NICs only**, so they are easily visible on the visualization

In [ ]:
$g.Vertices | 
    ? {$_.OriginalObject.GetType() -eq [Microsoft.Azure.Commands.Network.Models.PSNetworkInterface]} | 
    % {  $_.GVertexParameters.Fillcolor = [QuikGraph.Graphviz.Dot.GraphvizColor]::WhiteSmoke }

#### Exporting and visualizing

Now we just export the data into SVG and [Graphviz format](https://www.graphviz.org/doc/info/lang.html).

In [ ]:
Export-Graph -Graph $g -Path "$($env:TEMP)\topology.svg" -Format MSAGL_MDS
Export-Graph -Graph $g -Path "$($env:TEMP)\topology.gv" -Format Graphviz

In [ ]:
using System.IO;

var path = Path.GetTempPath();
var svg = File.ReadAllText($"{path}\\topology.svg");
var gv = File.ReadAllText($"{path}\\topology.gv");
svg.DisplayAs("text/html"); // this will embed a big SVG into the notebook, and it is not very convenient

So we are going to do it a bit differently. We will use HTML and [vis.js](https://visjs.org/). First we prepare a canvas

In [ ]:
#!html
<div id="mynetwork" style="height: 800px;"></div>

And then visualize the graph. pay attention to the line `#!share --from csharp gv`. It is a convenient way to "share" variables within the same kernel between different languages. In this case, we used C# to read the Graphviz file into the `gv` variable, and then use that variable within the block of JavaScript!

In [ ]:
#!js
#!share --from csharp gv

visRequire = interactive.configureRequire({
    paths: {
        visjs: "https://visjs.github.io/vis-network/standalone/umd/vis-network.min"
    }
});
    
visRequire(["visjs"], visjs => {
    
    var container = document.getElementById("mynetwork");
    var dot = gv;
    var parsedData = visjs.parseDOTNetwork(dot);

    var data = {
        nodes: parsedData.nodes,
        edges: parsedData.edges
    };
    var options = parsedData.options;
    options = {
        physics: {
            solver: "forceAtlas2Based",
            enabled: false,
            forceAtlas2Based: {
                theta: 0.5,
                gravitationalConstant: -50,
                centralGravity: 0.01,
                springConstant: 0.08,
                springLength: 100,
                damping: 0.4,
                avoidOverlap: 0.7
              },
            barnesHut: {
                theta: 0.5,
                gravitationalConstant: -2000,
                centralGravity: 0.3,
                springLength: 95,
                springConstant: 0.04,
                damping: 0.09,
                avoidOverlap: 0
            }
        },
        interaction: { hover: true, zoomView: true },
        layout: { randomSeed: 'Mickey' }
    }

    var network = new visjs.Network(container, data, options); 
    network.stabilize(600)
});

Voila! Here we see, that the misconfigured VNET is marked red. And it has a NIC attached to it. Which is, potentially, the root cause of the hypothetical problem :)